<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Introduction to Programming 2477-2223 S1
## Nova School of Business and Economics


### Team: Daniel Brosko (55036), ... (TODO)

TODO: theoretical description

## 1 Introduction

TODO

## 2 General packages loading

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
# from datetime import datetime
import datetime
import time
import yfinance as yf


#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp

# pip list # used for requirements.txt to get the version nubmers

In [2]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")

## 3 Market Capitalization Data Scraping

Here we scrape the market capitalizations of the firms using own scraping function from scraping_function.py.
The data are in billions of USD.

In [3]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

# mkt_cap_df_list

Elapsed time: 5.72 seconds.


In [4]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks
df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names
df_mkt_cap.set_index('Date', inplace=True)

df_mkt_cap

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/2554157159.py:5: 

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-04-01,0.78,0.00,6.24,2.04,1.84,1.07,31.68,4.98,4.81,12.40,27.33,13.47,9.12,1.17,14.97,6.26,94.12,6.70,1.93,7.62
2009-04-02,0.84,0.00,6.60,2.07,1.69,1.11,32.90,5.38,4.89,12.81,28.56,13.66,8.65,1.23,15.07,6.91,92.64,7.07,2.00,8.08
2009-04-03,0.89,0.00,6.45,2.13,1.79,1.10,33.69,5.52,4.96,12.64,29.27,13.57,8.76,1.27,14.52,7.01,91.16,7.25,2.07,8.12
2009-04-06,0.88,0.00,6.32,2.16,1.66,1.09,33.61,5.47,5.00,12.75,27.93,13.38,8.79,1.21,14.32,6.92,92.24,7.05,1.93,7.96
2009-04-07,0.83,0.00,5.95,2.14,1.79,1.06,32.54,5.41,4.86,12.58,26.43,13.31,8.50,1.17,15.03,6.61,90.90,6.72,1.80,7.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,13.95,54.11,186.35,34.04,6.46,16.63,941.85,19.86,53.78,63.71,66.26,22.95,51.39,4.32,42.08,64.60,277.49,41.83,24.81,12.20
2022-11-30,14.61,54.70,189.91,35.45,6.61,17.46,983.84,20.23,54.72,64.81,67.45,23.65,51.86,4.47,44.07,65.12,281.08,42.30,25.06,12.53
2022-12-01,14.43,51.23,190.35,36.56,6.57,18.01,973.24,20.31,54.25,66.03,67.39,23.66,51.33,4.90,43.59,65.08,286.41,42.57,24.66,12.34


## 4 Getting stock prices

In [5]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 15.302 seconds.


In [6]:
# loading stock prices into list of dataframes using own function
stocks_df_list = sp.load_stocks(success_tickers)
# stocks_df_list

In [7]:
df_close_prices = stocks_df_list[0][['Date','Adj Close']]
 
for index in range(1, len(stocks_df_list)):
    dataframe = stocks_df_list[index][['Date','Adj Close']]
    df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')

cp_columns_names = ['Date']
cp_columns_names.extend(success_tickers)
df_close_prices.columns = cp_columns_names
df_close_prices['Date'] = pd.to_datetime(df_close_prices['Date']).dt.date
df_close_prices.set_index('Date', inplace=True)

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_71779/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn

In [8]:
# Making returns from daily prices
daily_returns = df_close_prices.pct_change(1) # 1 for ONE DAY lookback
daily_returns.dropna(inplace=True)
daily_returns.head()

# monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
# annual_return = prices.pct_change(252)

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2017-12-08,0.005211,0.004841,0.010013,0.004944,0.012244,0.000201,0.001906,0.019543,0.025361,0.029153,0.008379,0.004062,0.027836,0.012488,0.019401,0.000388,0.006761,0.002810,-0.000987,0.010982
2017-12-11,-0.001234,0.009396,0.008475,-0.005418,-0.017444,-0.005520,0.005955,-0.018458,-0.015255,-0.010477,-0.001626,0.006426,-0.018396,-0.006204,0.006937,0.006211,0.013151,0.012259,-0.002305,-0.004639
2017-12-12,0.012111,-0.003103,0.006501,-0.002693,0.005184,-0.000807,-0.003285,0.004236,-0.001537,0.004314,0.004161,-0.015606,0.003443,-0.012039,-0.000854,-0.000129,0.010218,0.006228,0.004951,0.000341
2017-12-13,-0.002930,-0.041178,0.005514,-0.006718,-0.022174,-0.002626,-0.000815,-0.010392,-0.003808,0.003905,-0.016216,0.000721,-0.003994,-0.002482,0.009031,-0.002443,0.001640,-0.000688,-0.016749,-0.002614
2017-12-14,0.004653,-0.011985,-0.019897,-0.018647,-0.001977,-0.005064,0.008702,-0.004471,-0.018177,-0.003501,-0.008242,-0.007922,-0.009856,-0.006937,-0.021650,-0.014567,-0.004913,0.021335,-0.009018,-0.022559


In [9]:
# continue work here

### 5 Market Data from prof. Kenneth French's website 

We dowloaded data about the market from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html , however, we need to change the date into suitable format.

In [11]:
data_market = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3)
data_market = data_market[:-1]

data_market.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
data_market['Date'] = data_market['Date'].map(date_format_lambda)
data_market['Date'] = pd.to_datetime(data_market['Date']).dt.date
data_market.set_index('Date', inplace=True)
data_market.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009


In [12]:
# Filtering data for reasonable time interval
mkt_cap = df_mkt_cap.query("Date >= datetime.date(2018,1,1) and Date <= datetime.date(2022,10,31)")
returns = daily_returns.query("Date >= datetime.date(2018,1,1) and Date <= datetime.date(2022,10,31)")
market = data_market.query("Date >= datetime.date(2018,1,1) and Date <= datetime.date(2022,10,31)")

In [15]:
# adjusting stock returns to risk-free rate
risk_free_rate = market.loc[:, 'RF']
risk_free_rate = pd.DataFrame(risk_free_rate)
risk_free_rate_column = risk_free_rate.copy()

for column in range(1, len(success_tickers)):
    risk_free_rate = pd.concat([risk_free_rate, risk_free_rate_column], axis=1)

rf_returns = returns.subtract(risk_free_rate.values)

rf_returns

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.049171,-0.005751,0.002345,0.003144,0.007529,0.002412,0.010708,0.000931,0.030252,0.013766,0.002399,0.008867,0.029382,0.006827,0.042969,-0.007898,0.000074,0.015318,-0.003034,0.001296
2018-01-03,-0.001699,-0.013733,-0.012715,0.020868,-0.010621,-0.004794,0.006775,0.002064,0.012374,0.005076,0.003809,-0.017017,0.011501,-0.004273,0.010997,0.029252,0.001409,0.021617,-0.003042,-0.008786
2018-01-04,-0.001267,0.010843,-0.015590,0.005787,0.011795,-0.006301,-0.001524,-0.000244,0.009128,-0.015390,0.001698,-0.020077,0.009785,0.009875,-0.005588,-0.016656,-0.003821,0.011188,-0.002396,-0.003429
2018-01-05,0.017105,-0.012675,0.019397,0.010101,-0.003973,0.006551,0.010163,-0.012984,0.004090,0.019671,0.000184,-0.013698,0.012706,0.005809,0.001353,0.001676,-0.004097,-0.008765,-0.003388,-0.007672
2018-01-08,0.006717,-0.008240,0.001740,-0.015124,-0.007896,-0.000347,0.008425,-0.005414,-0.007076,-0.012161,-0.004554,0.005511,-0.017631,-0.008446,-0.043490,0.001863,-0.017123,-0.002303,-0.006326,0.007509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.017273,0.009833,-0.001597,0.010482,0.013874,0.042208,-0.004490,-0.001963,-0.001540,-0.003191,0.010396,0.012549,0.002445,0.049143,-0.014860,0.020645,-0.009902,0.013116,0.017513,-0.007188
2022-10-26,0.021643,0.015882,-0.009507,0.005887,-0.027686,0.009826,-0.051962,-0.008879,-0.008770,0.000864,-0.001955,-0.011286,-0.017644,-0.004269,0.014113,-0.014629,-0.000691,-0.001581,-0.006474,0.006932
2022-10-27,-0.040058,-0.005230,-0.012491,-0.017028,-0.000587,-0.141385,-0.051636,-0.007315,0.034996,-0.016025,-0.018313,-0.008040,0.029472,-0.027237,-0.020129,-0.002960,-0.017947,-0.000114,-0.015506,-0.007891


Just to remind what objects are important fot us now, we obtained the following dataframes that we are going to use:

* mkt_cap - market capitalization    
* rf_returns - daily stock returns adjusted to risk-free rate
* market - FF market factors

### 6 Market Betas

We will estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression.

In [ ]:
for (i in 1:length(Stock_Prices)) {if(dim(stock_adj_returns_and_market[[i]])[1] > 253){
                                  for (k in 254:dim(stock_adj_returns_and_market[[i]][1])[1]){
                                    betas[[i]][k] <- as.numeric(coef(lm(stock_adj_returns_and_market[[i]][(k-253):k,3] ~ stock_adj_returns_and_market[[i]][(k-253):k,2]))[2])                                          }
                                                                                       }
                                   }

In [ ]:
#due to few technical difficulties, we use "beta_frame" list to keep the correct index and prevent unwanted changes.
beta_frame <- Stock_Returns_Daily
for (i in 1:length(Stock_Returns_Daily)) {beta_frame[[i]] <- cbind(beta_frame[[i]], beta_x[[i]])}
for (i in 1:length(Stock_Returns_Daily)) {colnames(beta_frame[[i]])[2] <- 'Beta'}
for (i in 1:length(Stock_Returns_Daily)) {beta_frame[[i]] <- beta_frame[[i]][,2]}
tail(beta_frame[[4]])

In [ ]:
returns

In [ ]:
market

### 6 Book to Value Ratio Scraping

In [ ]:
# TODO - download book to market ratio

In [ ]:
# TODO - download market indeces

In [ ]:
# TODO - perform regression

In [ ]:
# TODO - put results into table

In [ ]:
# TODO - comment on findings

In [ ]:
# todo - solve issues

tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
# tickers.head()

tickers = tickers.Ticker
# tickers.head()

long_names = pd.DataFrame(index = range(len(tickers)), columns = 2)

for i in range(len(tickers)):
    long_names[i,0] = tickers[i]
    header = str(curlGetHeaders("https://www.macrotrends.net/stocks/charts/" + tickers[i])[5])
    long_names[i,1] = "Location: https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + header
    long_names[i,1] = gsub("/\r\n","",long_names[i,2])
    
    
long_names

#stocks <- pd.Data.Frame()

In [ ]:
# url = https://www.macrotrends.net/stocks/charts/AAPL/apple/price-book
#page = requests.get('https://www.macrotrends.net/stocks/charts/AAPL')#/apple/price-book')


In [ ]:
get.packages('rvest')
library(rvest)

stocks <- pd.Data.Frame(read.csv("tickers_set.csv", sep=","))
colnames(stocks) <- c("Index","Ticker") #maybe pridat adresu
head(stocks,3)

stock_names <- rep(NA,length(stocks$Ticker))
for (i in seq(1:length(stocks$Ticker))){
  stock_names[i] <- as.character(stocks[i,2])
}

long_names <- data.frame(matrix(NA, nrow = length(stocks$Ticker), ncol = 2))
for (i in seq(1:length(stocks$Ticker))){
  long_names[i,1] <- stock_names[i]
  long_names[i,2] <- gsub(paste0("Location: https://www.macrotrends.net/stocks/charts/", stock_names[i],"/"),"",as.character(curlGetHeaders(paste0("https://www.macrotrends.net/stocks/charts/",stock_names[i]))[5]))
  long_names[i,2] <- gsub("/\r\n","",long_names[i,2])
}
names(long_names) <- c("Ticker", "Long_Name")
print(long_names)
write.csv(long_names,"long_names.csv")


In [ ]:
my_date = datetime.strptime('2018-01-05', '%Y-%m-%d')
my_date.strftime('%A')